## Transformer

自注意力同时具有并行计算和最短的最大路径长度这两个优势。

所有的输入位置（tokens）是同时处理的。以Transformer为例：
 - [x1, x2, ... , xn], 我们一次性就计算出所有位置之间的注意力分数（注意力矩阵是一个NxN的矩阵）
 - 这一位在一次前向传播中，每个位置都能同时看到其他位置的信息，而不是一个接一个

优势：
- 充分利用GPU的并行处理能力，加快训练速度。
- 相比RNN，一个时刻一个时刻处理的方式，自注意力的效率更高。

信息从一个位置传播到另一个位置所需的路径长度的度量。
- 在RNN中，如果第一个词想影响最后一个词，中间要经过n步传递。
- 在CNN中，虽然可以并行，但要堆积多层卷积核才能覆盖整个输入。
- 在自注意力中，任何两个位置之间的信息交流都只需要一层，因为每个位置都能直接访问其他所有位置的表示。

优势：
- 使得长距离依赖问题更容易建模。
- 信息在序列传播的时间是最短的。

### Transformer劣势

1. 计算复杂度高，对于长序列（1000字以上），计算和显存开销剧增，难以训练。
2. 缺乏归纳偏置（Inductive bias）
    - 自注意力结构式全连接的，对位置不敏感的。对于某些任务（图像识别，语音识别），这种设计会让模型更依赖大量数据学习这些结构信息。
3. 训练和推理成本高
4. 对长序列建模能力虽强但不稳

### 模型

编码器和解码器架构

Transformer的编码器和解码器是基于自注意力的模块叠加而成的，源（输入）序列和目标（输出）序列的embedding表示加上位置编码，再分别输入到编码器和解码器中。

Transformer的编码器是由多个相同的层叠加而成的，每个层都有两个子层。第一个子层是多头自注意力汇聚；第二个子层是基于位置的前馈网络。受残差网络的影响，每个子层都采用了残差连接。在残差连接的计算之后，紧接着应用层规范化。因此，输入序列对应的每个位置，Transformer编码器豆浆输出一个d维表示向量。


解码器在这两个子层之间插入了第三个子层，称为编码器-解码器注意力。在编码器-解码器注意力中，查询来自前一个解码器层的输出，而键和值来自整个编码器的输出。在解码器自注意力中，查询、键和值都来自上一个解码器层的输出。但是解码器中每个位置只能考虑该位置之前的所有位置。这种掩蔽（masked）注意力保留了自回归属性。确保预测仅依赖于已生成的输出词元。

In [2]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

层规范化和批量规范化的目标相同，但层规范化是基于特征维度进行规范化。自然语言处理中，输出通常是变长序列，批量规范化通常不如层规范化的效果好。

In [4]:
#@save
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)
    

### 编码器

In [ ]:
#@save
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens,
            num_heads,dropout,use_bias
        )
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.fnn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)
    
    def forward(self, X, valid_lens):
        # 这里的三个 X 是传给多头注意力（MultiHeadAttention）模块的三个输入：query、key、value。这是 Transformer 自注意力（self-attention）机制的标准用法。
        Y = self.addnorm1(X, self.attention(X,X,X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [8]:
#@save
class TransformerEncoder(d2l.Encoder):
    """Transformer编码器"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[
                i] = blk.attention.attention.attention_weights
        return X